# Introduction au Machine Learning avec ML.NET

## Qu'est-ce que le Machine Learning ?

Le Machine Learning (ML) est une méthode de création de modèles prédictifs en utilisant des algorithmes pour apprendre à partir de données. Contrairement à la programmation traditionnelle, où les développeurs définissent les étapes de l'algorithme, le ML permet à la machine d'apprendre ces étapes à partir des données.


## Quelles sont les étapes à suivre ?

1. **Source et Préparation des Données d'Entraînement**  
   Pour entraîner le modèle, nous avons besoin de données étiquetées. Les données étiquetées signifient simplement un ensemble de données qui contient la colonne à prédire déjà présente afin que l'algorithme d'entraînement puisse apprendre à prédire les valeurs.

2. **Choisir l'Algorithme d'Entraînement et Entraîner**  
    >**Spoiler Alert**  
    >Dans la plupart de nos exemples, nous utiliserons AutoML pour simplifier ce processus. AutoML essaie stratégiquement divers algorithmes et paramètres pour une tâche donnée afin de trouver celui qui fonctionne le mieux pour vos données !  
    >  
    >Vous pouvez penser à cela comme une boucle qui essaie toutes les options. Notre AutoML est un peu plus intelligent que cela ... mais c'est essentiellement ce qu'il fait !  
    >
    > Pour l'exemple ci-dessous, nous entraînerons un algorithme spécifique - pour que vous puissiez voir comment cela fonctionne !  
    1. Choisir une Tâche - [Tâches ML.NET](https://docs.microsoft.com/dotnet/machine-learning/resources/tasks)
    2. Choisir un Algorithme - [Algorithmes ML.NET](https://docs.microsoft.com/dotnet/machine-learning/how-to-choose-an-ml-net-algorithm)
    3. Définir les Paramètres de l'Algorithme [Glossaire - Hyperparamètres](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary#hyperparameter)
    4. Entraîner - C'est ici que les données sont réellement alimentées dans l'algorithme pour entraîner le modèle. Cela peut prendre du temps en fonction de la quantité de données, de l'algorithme et des paramètres de cet algorithme.

3. **Évaluer**  
   Une fois que vous avez entraîné un modèle - comment savez-vous qu'il fonctionne ? Il existe de nombreuses techniques pour évaluer les performances de vos modèles. Si vous souhaitez approfondir - consultez [Les Métriques d'Évaluation](https://docs.microsoft.com/dotnet/machine-learning/resources/metrics). Sinon, nous donnerons des exemples tout au long de ces tutoriels.

4. **Déployer**  
   Après avoir entraîné un modèle ... c'est juste du code .NET ! Déployez-le comme vous le feriez pour toute autre application.

## Comment commencer ?

Ci-dessous, nous avons une introduction rapide à ML.NET - "Hello ML.NET World!" et les trois prochains notebooks de la série plongent en profondeur dans la [Préparation des Données et l'Ingénierie des Caractéristiques](https://ntbk.io/ml-02-data), [L'Entraînement et AutoML](https://ntbk.io/ml-03-training), et [L'Évaluation des Modèles](https://ntbk.io/ml-04-evaluation).

# Hello ML.NET World!

Le code dans l'extrait suivant démontre l'application ML.NET la plus simple. Cet exemple construit un modèle de régression linéaire pour prédire les prix des maisons en utilisant les données de taille et de prix des maisons. 

Première étape, référencer le package [Microsoft.ML](https://www.nuget.org/packages/Microsoft.ML/).

In [18]:
#r "nuget: Microsoft.ML"

Installed Packages Microsoft.ML, 3.0.1

La deuxième étape est de référencer les espaces de noms ML.NET.

In [19]:
using Microsoft.ML;
using Microsoft.ML.Data;

Maintenant, nous sommes prêts à écrire le code pour accomplir la tâche de machine learning dont nous avons besoin. Toujours commencer par créer le [MLContext](https://docs.microsoft.com/dotnet/api/microsoft.ml.mlcontext?view=ml-dotnet) qui est le contexte commun pour toutes les opérations ML.NET.

In [20]:
MLContext mlContext = new MLContext();

Ensuite, définissez les structures de données pour les données que nous allons utiliser. Cet exemple concerne la prédiction des prix des maisons. Commencez par définir la structure de données suivante qui contient la taille et le prix des maisons :

In [21]:
public class HouseData
{
    public float Size { get; set; }
    public float Price { get; set; }
}

Définissez ensuite la structure de données pour les prédictions de prix des maisons :

In [22]:
public class Prediction
{
    [ColumnName("Score")]
    public float Price { get; set; }
}

Maintenant, nous sommes prêts à entraîner les données pré-collectées que nous utiliserons pour le scénario de prédiction des prix des maisons :

In [23]:
HouseData[] houseData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.4F, Price = 3.7F }
};

IDataView trainingData = mlContext.Data.LoadFromEnumerable(houseData);

En utilisant le `MLContext` que nous avons précédemment créé, chargez les données d'entraînement dans le [IDataView](https://docs.microsoft.com/dotnet/api/microsoft.ml.idataview?view=ml-dotnet) ML.NET, qui est le type de données fondamental de ML.NET.

In [24]:
IDataView trainingData = mlContext.Data.LoadFromEnumerable(houseData);

Maintenant que nous avons les données prêtes, nous allons créer le pipeline ML.NET en spécifiant le formateur que nous allons utiliser pour construire notre modèle de machine learning. Pour la prédiction des prix des maisons, nous allons utiliser le formateur de régression. ML.NET prend en charge d'autres formateurs de machine learning qui peuvent être utilisés pour d'autres scénarios selon les besoins. Le pipeline créera ce que l'on appelle un [Estimator](https://docs.microsoft.com/dotnet/api/microsoft.ml.iestimator-1?view=ml-dotnet), utilisé pour définir les opérations appliquées aux données.

In [25]:
var pipeline = mlContext.Transforms.Concatenate("Features", new[] { "Size" })
               .Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "Price", maximumNumberOfIterations: 100));

Après avoir créé l'estimateur, nous sommes prêts à appliquer les transformations et le formateur définis dans le pipeline aux données. Pour ce faire, appelez la méthode [Fit](https://docs.microsoft.com/dotnet/api/microsoft.ml.iestimator-1.fit?view=ml-dotnet).


In [26]:
var model = pipeline.Fit(trainingData);

Maintenant, nous pouvons évaluer le modèle entraîné. Pour ce faire, nous chargerons des données de test préparées et ensuite nous appellerons la méthode [Evaluate](https://docs.microsoft.com/dotnet/api/microsoft.ml.regression

catalog.evaluate?view=ml-dotnet), puis nous afficherons le [Coefficient de Détermination](https://en.wikipedia.org/wiki/Coefficient_of_determination) pour savoir comment le modèle s'ajuste aux données de test. Plus le coefficient de détermination est proche de 1, mieux le modèle est ajusté. Répétez les étapes d'entraînement et d'évaluation jusqu'à obtenir un résultat satisfaisant du modèle entraîné.

In [27]:
HouseData[] testData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.2F, Price = 1.5F },
    new HouseData() { Size = 1.4F, Price = 1.7F },
    new HouseData() { Size = 1.6F, Price = 1.9F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.2F, Price = 3.5F },
    new HouseData() { Size = 3.3F, Price = 3.6F },
    new HouseData() { Size = 3.5F, Price = 3.9F },
    new HouseData() { Size = 3.7F, Price = 4.3F },
    new HouseData() { Size = 4.0F, Price = 6.1F },
    new HouseData() { Size = 5.0F, Price = 7.2F },
    new HouseData() { Size = 6.0F, Price = 8.5F },
    new HouseData() { Size = 7.0F, Price = 9.8F },
    new HouseData() { Size = 8.0F, Price = 10.3F }
    };

IDataView trainingTestData = mlContext.Data.LoadFromEnumerable(testData);
IDataView transformedTestData = model.Transform(trainingTestData);
RegressionMetrics trainedModelMetrics = mlContext.Regression.Evaluate(transformedTestData, labelColumnName: "Size");
Console.WriteLine($"Coefficient de détermination pour le modèle entraîné : {trainedModelMetrics.RSquared:0.00}");

Coefficient de détermination pour le modèle entraîné : 0,97


Maintenant, nous avons le modèle entraîné prêt pour la prédiction. Utilisons ce modèle pour prédire le prix d'une maison. Pour ce faire, créez l'objet [PredictionEngine<TSrc,TDst>](https://docs.microsoft.com/dotnet/api/microsoft.ml.predictionengine-2?view=ml-dotnet). Le moteur de prédiction est la classe utilisée pour faire des prédictions uniques sur un modèle précédemment entraîné (et le pipeline de transformation précédent). La création du moteur de prédiction à partir du modèle entraîné peut se faire avec le code suivant :


In [28]:
var predictionEngine = mlContext.Model.CreatePredictionEngine<HouseData, Prediction>(model);

Ensuite, en utilisant le moteur de prédiction créé, nous pouvons prédire le prix de la maison comme suit :

In [29]:
var size = new HouseData() { Size = 2.5F };
var price = predictionEngine.Predict(size);
Console.WriteLine($"Prix prédit pour une taille de {size.Size * 1000} pieds carrés : {price.Price * 100:C}k");

Prix prédit pour une taille de 2500 pieds carrés : 276,29 ¤k


Félicitations ! Vous avez entraîné avec succès un modèle de régression ML.NET en utilisant vos propres données, puis utilisé ce modèle pour prédire les prix des maisons. Voici un diagramme résumant l'opération de bout en bout de création et d'entraînement d'un modèle ML.NET, puis de l'utiliser pour prédire les prix des maisons.

![ML.NET Workflow](https://docs.microsoft.com/dotnet/machine-learning/media/mldotnet-annotated-workflow.png)

# Continuez à apprendre

> [⏩ Module Suivant - Préparation des Données et Ingénierie des Caractéristiques](./02-Data%20Preparation%20and%20Feature%20Engineering.ipynb)